In [1]:
# Import all required packages.

from IPython.display import display
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
import torch.optim as optim

In [2]:
# run the notebok to get data.

%run STRAIN_VariableGenerator.ipynb

In [3]:
# Define ModelClass

class Strain(nn.Module):

    def __init__(self, num_features, input_seq_len_list, input_size=2, num_classes=5, hidden_dim = 100):
        super(Strain, self).__init__()
        
        time_steps = input_seq_len_list[0]
        batch_size = 1
        self.rnn1 = nn.LSTM(input_size, hidden_dim, 1) # 1 is number of layers.
        self.linear = nn.Linear(hidden_dim, num_classes)

    def forward(self, x, indices):
        y_out, _ = self.rnn1(x)
        y_out = torch.index_select(y_out, 0, indices)
        y_out = y_out.view(y_out.shape[0], -1)
        y_out = self.linear(y_out)
        
        return y_out

#     def num_flat_features(self, x):
#         size = x.size()[1:]  # all dimensions except the batch dimension
#         num_features = 1
#         for s in size:
#             num_features *= s
#         return num_features


In [4]:
# input_seq
# target
# indices

input_seq, target, indices = generate_variables(feature_list=["activity_details"], restrict_seqlen=10)["activity_details"]

# declaring Network.
net = Strain(num_features=1, input_seq_len_list=[len(input_seq)])
criterion = nn.CrossEntropyLoss(size_average=True)
optimizer = optim.SGD(net.parameters(), 0.001)

In [ ]:
# Train the network

for i in range(2):
    optimizer.zero_grad()
    y_hat = net.forward(input_seq, indices=indices)
    loss = criterion(y_hat, target)
    loss.backward()
    optimizer.step()
    
    print(loss)
    

In [ ]:
print(indices)
print(input_seq.shape)

In [ ]:
x = torch.randn(300, 1, 100)
x = Variable(x)
y = torch.LongTensor([1,3,10])
y = Variable(y)
x = torch.index_select(x, 0, y)
print(x)

In [ ]:
print(y_hat)

print(target)